In [30]:
#from the schedule in the file '23_24_schedule' create a dataframe that has all of the games for the current day
import os
import pandas as pd
import datetime

current_date = datetime.date.today()
print(current_date)
df = pd.read_csv('23_24_sched.csv')
df['Date'] = pd.to_datetime(df['Date']).dt.date
df2 = df[df['Date'] == current_date]
td = df2[['Game','AwayTeam','HomeTeam']]
td.head()
td.to_csv("today.csv", index=False)

2023-12-06


In [33]:
#do the rolling average average but in order to get the data for the last 10 games, we perform it wihtout  shifting by one to include the current games
#then take the last row from each team's files and create a new file that has the most recent stats for every team so pull from if they play today
import os
import pandas as pd
import datetime

current_date = pd.Timestamp(datetime.date.today())

def calculate_rolling_average(df, columns, window=10):
    rolling_df = df[columns].rolling(window=window, min_periods=10).mean()
    return rolling_df

#Base path for the original data 
base_path = '23_24_Team'
df4 = pd.DataFrame()
# New top-level folder for averaged data
average_data_path = '23_24_test'
os.makedirs(average_data_path, exist_ok=True)
# Iterate over each year's folder
# Iterate over each team's CSV file
for team_file in os.listdir(base_path):
    if team_file.endswith('.csv'):
        team_path = os.path.join(base_path, team_file)
        # Read the CSV file
        df = pd.read_csv(team_path)

        # Select the desired columns and calculate the rolling averages
        selected_columns = ['Game', 'Team', 'Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore', 'HomeResult', 'days_of_rest']
        other_columns = df.columns.difference(selected_columns)
        rolling_df = calculate_rolling_average(df, other_columns)

    
        # Combine the selected columns with the rolling averages
        combined_df = pd.concat([df[selected_columns], rolling_df], axis=1)
        cols_to_drop = ['Game','AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore', 'HomeResult','days_of_rest']
       
    #drop other unneccesary columns
        combined_df.drop(columns=cols_to_drop, inplace=True)
        combined_df['Date'] = pd.to_datetime(combined_df['Date'])
        combined_df['time_diff']=(current_date-combined_df['Date'])
        combined_df['days_of_rest'] = combined_df['time_diff'].dt.days - 1
        combined_df.drop('time_diff', axis=1, inplace=True)
        combined_df.insert(1, 'days_of_rest', combined_df.pop('days_of_rest'))
        last_row_df = combined_df.tail(1)
        df4 = pd.concat([df4, last_row_df], ignore_index=True, axis=0)
        # Save the new DataFrame as a CSV file in the year-specific folder under 'average_data'

#df4.head()
df4.to_csv('test.csv', index=False)

In [34]:
#pull form the file we jsut created and the list of today's game and get the stats for the home and away teams with the prefixes added and save it to the "todays_games.csv"
df_test = pd.read_csv('test.csv')
df_today= pd.read_csv('today.csv')

def add_prefix_to_columns(df, prefix):
    df.columns = [prefix + col for col in df.columns]
    return df

def find_team_data_modified(team, df_test):
    team_data = df_test[df_test['Team'] == team].drop(columns=['Team', 'Date'])
    return team_data
combined_data = []
for index, row in df_today.iterrows():
    game_info = row['Game']
    away_team = row['AwayTeam']
    home_team = row['HomeTeam']
    away_team_data = add_prefix_to_columns(find_team_data_modified(away_team, df_test).copy(), 'a_')
    home_team_data = add_prefix_to_columns(find_team_data_modified(home_team, df_test).copy(), 'h_')
    if not away_team_data.empty:
        away_team_data = away_team_data.iloc[0]
    if not home_team_data.empty:
        home_team_data = home_team_data.iloc[0]

    combined_row = pd.concat([away_team_data, home_team_data])
    combined_row['Game'] = game_info
    combined_data.append(combined_row)
df_combined_corrected = pd.DataFrame(combined_data)
df_combined_corrected.to_csv('today_games.csv', index=False)